# Compare multivariate linear models computing time for dominance analysis

In [13]:
import pandas as pd
import numpy as np
import pathlib
import sys
import os 
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from IPython.display import display
from dominance_analysis import Dominance
from netneurotools.stats import get_dominance_stats
import contextlib

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.stats import dominance

/Users/llotter/projects/juspyce


## sample data

In [14]:
np.random.seed(42)
x = np.random.random([100,10]) * 10
y = np.random.random([100,1]) * 5
display(x.shape)
display(y.shape)

adj_r2 = False

(100, 10)

(100, 1)

## MLR

### sklearn

In [15]:
def get_r2(x,y, adj_r2=False):
    mlr = LinearRegression().fit(X=x, y=y)
    if adj_r2:
        return 1 - (1-mlr.score(X=x, y=y))*(len(y)-1)/(len(y)-x.shape[1]-1)
    else:
        return mlr.score(X=x, y=y)

print(get_r2(x,y, adj_r2=adj_r2))
%timeit get_r2(x,y, adj_r2=adj_r2)

0.07494045395314919
420 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### statsmodels

In [21]:
def get_r2(x,y, adj_r2=False):
    mlr = sm.OLS(y,sm.add_constant(x)).fit()
    if adj_r2:
        return mlr.rsquared_adj
    else:
        return mlr.rsquared

print(get_r2(x,y, adj_r2=adj_r2))
%timeit get_r2(x,y, adj_r2=adj_r2)

0.07494045395314908
360 µs ± 3.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### custom 1

In [17]:
def get_r2(X,y, adj_r2=False):
    X = np.c_[X, np.ones(X.shape[0])]
    beta = np.linalg.lstsq(X, y, rcond=None)[0]
    y_hat = np.dot(X, beta)
    ss_res = np.sum((y-y_hat)**2)       
    ss_tot = np.sum((y-np.mean(y))**2)  
    r2 = 1 - ss_res / ss_tot
    if adj_r2:
        return 1 - (1-r2) * (len(y)-1) / (len(y)-x.shape[1]-1)
    else:
        return r2

print(get_r2(x,y, adj_r2=adj_r2))
%timeit get_r2(x,y, adj_r2=adj_r2)

0.07494045395314886
207 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### custom 2

In [20]:
def get_r2(X, y, adj_r2=False):
    X = np.c_[X, np.ones(X.shape[0])] 
    beta = np.linalg.pinv((X.T).dot(X)).dot(X.T.dot(y))
    y_hat = np.dot(X, beta)
    ss_res = np.sum((y-y_hat)**2)       
    ss_tot = np.sum((y-np.mean(y))**2)   
    r2 = 1 - ss_res / ss_tot  
    if adj_r2:
        return 1 - (1-r2) * (len(y)-1) / (len(y)-x.shape[1]-1)
    else:
        return r2

print(get_r2(x,y, adj_r2=adj_r2))
%timeit get_r2(x,y, adj_r2=adj_r2)

0.07494045395314908
111 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Dominance analysis

Implementation of "custom 2" code

In [23]:
adjust_r2 = False

# custom version
def dominance_my(x, y):
    dom = dominance(x, y, adj_r2=adjust_r2, verbose=False)
    return dom["total"]

# dominance-analysis toolbox
def dominance_toolbox(x, y):
    X = pd.DataFrame(x, columns=[f"pred{i}" for i in range(x.shape[1])])
    Y = pd.DataFrame(y, columns=["target"])
    df=pd.concat([X,Y], axis=1)
    with contextlib.redirect_stdout(None):
        dom = Dominance(
            data=df, 
            target="target",
            objective=1,
            top_k=x.shape[1])
        dom.incremental_rsquare()
        dom_stats = dom.dominance_stats()
    return dom_stats["Total Dominance"].sort_index().values
    
# network neuroscience lab tool
def dominance_netneuro(x, y):
    dom, _ = get_dominance_stats(X=x, y=y, use_adjusted_r_sq=adjust_r2, verbose=False)
    return dom["total_dominance"]

### compare

In [24]:
print("Custom")
print(dominance_my(x,y))
%timeit dominance_my(x,y)

Custom
[0.00290934 0.04157522 0.00768184 0.00031752 0.00255071 0.00342477
 0.0061324  0.00217687 0.00502667 0.0031451 ]
95.1 ms ± 805 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
print("netneurotools")
print(dominance_netneuro(x,y))
%timeit dominance_netneuro(x,y)

netneurotools
[0.00290934 0.04157522 0.00768184 0.00031752 0.00255071 0.00342477
 0.0061324  0.00217687 0.00502667 0.0031451 ]
367 ms ± 5.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
print("dominance-analysis toolbox")
print(dominance_toolbox(x,y))
%timeit dominance_toolbox(x,y)

  0%|          | 0/10 [00:00<?, ?it/s]

dominance-analysis toolbox


 30%|███       | 3/10 [00:00<00:00, 10.57it/s]

[0.0029093428978056953 0.041575219681253404 0.007681842235021487
 0.0003175244846935784 0.0025507064295806395 0.003424772808622289
 0.006132398551865958 0.0021768737525747735 0.0050266687085035195
 0.003145104403227845]


100%|██████████| 9/9 [00:00<00:00, 48.29it/s]

1.88 s ± 14.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
